# 0. Tools and Python (3.7.4) packages   
**bedtools v2.31.0**  
**pandas v1.3.5**   
**NumPy v1.21.5**  
**Bio v1.5.9** 

# Resources   
1. [UCSC Genome Browser](http://genome.ucsc.edu/index.html)   
2. [NCBI](https://www.ncbi.nlm.nih.gov/)




In [1]:
import pandas as pd
import numpy as np
from Bio import Entrez
Entrez.email = "an.eudokimova@yandex.ru"

****************

# 1. Task1. Annotation of input BED-file   
1.1 Download GTF files for the main gene transcript sets from [[1]]. The BED-file header states that reference=hg19:     
```bash
wget https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/genes/hg19.ncbiRefSeq.gtf.gz
```   
```bash 
gunzip hg19.ncbiRefSeq.gtf.gz
```   
   
Based on the wording of the task, we are only interested in exons, so we extract only exons from GTF-file:   
```bash 
grep -w 'exon' hg19.ncbiRefSeq.gtf > hg19.ncbiRefSeq.exon.gtf
```
   
1.2 To identify gene regions in input BED-file we will use bedtools intersect, this tool allows one to screen for overlaps between two sets of genomic features:   
``` bash 
bedtools intersect -a IAD143293_241_Designed.bed -b hg19.ncbiRefSeq.exon.gtf -loj -wa -wb > intersect.txt
```   
Options `-wa` and `-wb` allows to write the original entries of both A and B input files in output   
Option `-loj` report a NULL feature for B, if no overlaps are found   

Creat directory for final results:   
```bash 
mkdir RESULTS
```

[1]: https://hgdownload.soe.ucsc.edu/


1.3 Analisys of output file   

In [2]:
intersect = pd.read_csv('intersect.txt', sep='\t', header=None)
intersect.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,chr1,18722630,18722936,AMPL7162509462,.,Pool=1;SUBMITTED_REGION=AMPL7162509462,.,.,.,-1,-1,.,.,.,.
1,chr1,113456395,113456721,AMPL7166063003,.,Pool=2;SUBMITTED_REGION=AMPL7166063003,chr1,ncbiRefSeq.2021-05-17,exon,113454469,113456787,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_001166496..."
2,chr1,113456395,113456721,AMPL7166063003,.,Pool=2;SUBMITTED_REGION=AMPL7166063003,chr1,ncbiRefSeq.2021-05-17,exon,113454469,113456787,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_003051.4""..."
3,chr1,113459897,113460203,AMPL7166063010,.,Pool=1;SUBMITTED_REGION=AMPL7166063010,chr1,ncbiRefSeq.2021-05-17,exon,113459800,113460666,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_001166496..."
4,chr1,113459897,113460203,AMPL7166063010,.,Pool=1;SUBMITTED_REGION=AMPL7166063010,chr1,ncbiRefSeq.2021-05-17,exon,113459800,113460666,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_003051.4""..."


We can see, that not all amplicons are identified (-1 in columns 9 and 10).    
Extraction gene name and exon number from column 14:  

In [3]:
tmp = intersect[14].str.split(' ', expand=True).iloc[:, [9, 5]]. \
    reset_index().drop('index', axis=1).replace(np.nan, '.').replace([';', '"'], '', regex=True)
tmp.columns = [15, 16]
out = pd.concat([intersect, tmp], axis=1)
out.head(6)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,chr1,18722630,18722936,AMPL7162509462,.,Pool=1;SUBMITTED_REGION=AMPL7162509462,.,.,.,-1,-1,.,.,.,.,.,.
1,chr1,113456395,113456721,AMPL7166063003,.,Pool=2;SUBMITTED_REGION=AMPL7166063003,chr1,ncbiRefSeq.2021-05-17,exon,113454469,113456787,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_001166496...",SLC16A1,5
2,chr1,113456395,113456721,AMPL7166063003,.,Pool=2;SUBMITTED_REGION=AMPL7166063003,chr1,ncbiRefSeq.2021-05-17,exon,113454469,113456787,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_003051.4""...",SLC16A1,5
3,chr1,113459897,113460203,AMPL7166063010,.,Pool=1;SUBMITTED_REGION=AMPL7166063010,chr1,ncbiRefSeq.2021-05-17,exon,113459800,113460666,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_001166496...",SLC16A1,4
4,chr1,113459897,113460203,AMPL7166063010,.,Pool=1;SUBMITTED_REGION=AMPL7166063010,chr1,ncbiRefSeq.2021-05-17,exon,113459800,113460666,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_003051.4""...",SLC16A1,4
5,chr1,113460270,113460593,AMPL7163702396,.,Pool=2;SUBMITTED_REGION=AMPL7163702396,chr1,ncbiRefSeq.2021-05-17,exon,113459800,113460666,.,-,.,"gene_id ""SLC16A1""; transcript_id ""NM_001166496...",SLC16A1,4


We can see rows with the same amplicons, gene names and exon coordinates and number, but with different transcript_id in column 14. As far as we are not interested in transcript_id, we can remove this duplicates:

In [4]:
out = out.drop_duplicates([3, 15, 16], keep='first').reset_index().drop('index', axis=1)

...and finaly extract target columns, rename them and write output file:

In [5]:
out = out.iloc[:, [0,1, 2, 3, 4, 5, 15, 16]]
out.columns = ['#CHR', 'START_POS', 'END_POS', 'AMPL_ID', 'STRAND', 'AMPL_DESCRIPTION', 'GENE_NAME', 'EXON_NUMBER']
out.to_csv('RESULTS/task1_output.bed', sep='\t', index=None, header=True)
out.head(6)

,#CHR,START_POS,END_POS,AMPL_ID,STRAND,AMPL_DESCRIPTION,GENE_NAME,EXON_NUMBER
0,chr1,18722630,18722936,AMPL7162509462,.,Pool=1;SUBMITTED_REGION=AMPL7162509462,.,.
1,chr1,113456395,113456721,AMPL7166063003,.,Pool=2;SUBMITTED_REGION=AMPL7166063003,SLC16A1,5
2,chr1,113459897,113460203,AMPL7166063010,.,Pool=1;SUBMITTED_REGION=AMPL7166063010,SLC16A1,4
3,chr1,113460270,113460593,AMPL7163702396,.,Pool=2;SUBMITTED_REGION=AMPL7163702396,SLC16A1,4
4,chr1,113471727,113472054,AMPL7166063018,.,Pool=1;SUBMITTED_REGION=AMPL7166063018,SLC16A1,2
5,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,SLC16A1-AS1,1


Add header of input BED-file in output file:   


In [6]:
with open('IAD143293_241_Designed.bed', 'r') as input:
    header = input.readline()  
with open('RESULTS/task1_output.bed', 'r+') as output:
    content = output.read()
    output.seek(0, 0)  
    output.write(header)  
    output.write(content)  

1.4 Get information about genes

In [7]:
# function for searching gene ID, description and summary
def get_id(gene):
    try:
        handle = Entrez.esearch(db="gene",term=f"Homo[Orgn] AND {gene}[Gene]")
        result = Entrez.read(handle)
        to_return = result['IdList'][0]
    except:
        to_return = np.nan
    return to_return

def get_summary(ID):
    try:
        handle = Entrez.epost("gene", id=ID)
        result = Entrez.read(handle)
        webEnv = result["WebEnv"]
        queryKey = result["QueryKey"]
        data = Entrez.esummary(db="gene", webenv=webEnv, query_key=queryKey)
        result = Entrez.read(data)
        to_return = [result['DocumentSummarySet']['DocumentSummary'][0]['Description'],
            result['DocumentSummarySet']['DocumentSummary'][0]['Summary']]
    except:
        to_return = [np.nan, np.nan]
    return to_return

In [8]:
# create DF with gene names
genes = pd.DataFrame({'GENE_NAME':out['GENE_NAME'].unique()})
genes = genes[genes['GENE_NAME'] != '.'].reset_index().drop('index', axis=1)

# get ID, description and summary of each gene using Entrez
genes['ID'] = genes['GENE_NAME'].apply(get_id)
tmp = pd.DataFrame(list(map(get_summary, genes['ID'])))
tmp.columns = ['DESCRIPTION', 'SUMMARY']
genes = pd.concat([genes, tmp], axis=1)

genes.to_csv('RESULTS/genes_summary.csv', sep='\t', index=None)


We have 31 genes (see table below). Some functions of protein encoded by these genes:   
- transpoters (*SLC16A1; ABCC8*)   
- enzymes (*G6PC2; HADH*)   
- transcription regulatory factors or part of them (*KLF11 - cell growth inhibition and apoptosis; NEUROD1 - regulation expression of the insulin gene, ZFP57; RFX6; PAX4; GLIS3; NEUROG3 - involved in neurogenesis; HNF1A - regulation of expression of several liver-specific genes; HNF4A; PDX1 - activator of insulin; somatostatin; glucokinase etc; HNF1B - regulation of nephron and embryonic pancreas development; FOXP3*)   
- kinases (*GCK; BLK - cell proliferation and differentiation; GLUD1 - regulating amino acid-induced insulin secretion; INSR - insulin receptor*)   
- potassium channel (*KCNJ11*)    
    
Based on information about these genes, I can assume, that this targeted panel under consideration can be used to diagnose and assess susceptibility to various types of diabetes, as well as other metabolic disorders associated with the functioning of the pancreas and liver.    
Some of these genes can be used for canser diagnostic (for example *BLK, HNF1B, PAX4*), but I don't think that it is a purpose of this pannel

In [9]:
pd.set_option('display.max_colwidth', None)
genes

,GENE_NAME,ID,DESCRIPTION,SUMMARY
0,SLC16A1,6566,solute carrier family 16 member 1,"The protein encoded by this gene is a proton-linked monocarboxylate transporter that catalyzes the movement of many monocarboxylates, such as lactate and pyruvate, across the plasma membrane. Mutations in this gene are associated with erythrocyte lactate transporter defect. Alternatively spliced transcript variants have been found for this gene.[provided by RefSeq, Oct 2009]"
1,SLC16A1-AS1,100506392,SLC16A1 antisense RNA 1,
2,KLF11,8462,KLF transcription factor 11,"The protein encoded by this gene is a zinc finger transcription factor that binds to SP1-like sequences in epsilon- and gamma-globin gene promoters. This binding inhibits cell growth and causes apoptosis. Defects in this gene are a cause of maturity-onset diabetes of the young type 7 (MODY7). Three transcript variants encoding two different isoforms have been found for this gene. [provided by RefSeq, Apr 2010]"
3,G6PC2,57818,glucose-6-phosphatase catalytic subunit 2,"This gene encodes an enzyme belonging to the glucose-6-phosphatase catalytic subunit family. These enzymes are part of a multicomponent integral membrane system that catalyzes the hydrolysis of glucose-6-phosphate, the terminal step in gluconeogenic and glycogenolytic pathways, allowing the release of glucose into the bloodstream. The family member encoded by this gene is found in pancreatic islets and does not exhibit phosphohydrolase activity, but it is a major target of cell-mediated autoimmunity in diabetes. Several alternatively spliced transcript variants of this gene have been described, but their biological validity has not been determined. [provided by RefSeq, Jul 2008]"
4,NEUROD1,4760,neuronal differentiation 1,"This gene encodes a member of the NeuroD family of basic helix-loop-helix (bHLH) transcription factors. The protein forms heterodimers with other bHLH proteins and activates transcription of genes that contain a specific DNA sequence known as the E-box. It regulates expression of the insulin gene, and mutations in this gene result in type II diabetes mellitus. [provided by RefSeq, Jul 2008]"
5,MFF-DT,654841,MFF divergent transcript,
6,HADH,3030,hydroxyacyl-CoA dehydrogenase trifunctional multienzyme complex subunit alpha,"This gene encodes the alpha subunit of the mitochondrial trifunctional protein, which catalyzes the last three steps of mitochondrial beta-oxidation of long chain fatty acids. The mitochondrial membrane-bound heterocomplex is composed of four alpha and four beta subunits, with the alpha subunit catalyzing the 3-hydroxyacyl-CoA dehydrogenase and enoyl-CoA hydratase activities. Mutations in this gene result in trifunctional protein deficiency or LCHAD deficiency. The genes of the alpha and beta subunits of the mitochondrial trifunctional protein are located adjacent to each other in the human genome in a head-to-head orientation. [provided by RefSeq, Jul 2008]"
7,ZFP57,346171,ZFP57 zinc finger protein,"The protein encoded by this gene is a zinc finger protein containing a KRAB domain. Studies in mouse suggest that this protein may function as a transcriptional repressor. Mutations in this gene have been associated with transient neonatal diabetes mellitus type 1 (TNDM1).[provided by RefSeq, Sep 2009]"
8,RFX6,222546,regulatory factor X6,"The nuclear protein encoded by this gene is a member of the regulatory factor X (RFX) family of transcription factors. Studies in mice suggest that this gene is specifically required for the differentiation of islet cells for the production of insulin, but not for the differentiation of pancreatic polypeptide-producing cells. It regulates the transcription factors involved in beta-cell maturation and function, thus, restricting the expression of the beta-cell differentiation and specification genes. Mutations in this gene are associated with Mitchell-Riley syndrome, which is characterized by neonatal diabetes with pancreatic hypoplasia, duodenal and jejun

*********************

# 2. Task2. Searching of off-target region in the pannel    
2.1. First of all, let's **repeat BED-file annotation** whithout "exon" filter:   
```bash
bedtools intersect -a IAD143293_241_Designed.bed -b hg19.ncbiRefSeq.gtf -loj -wa -wb > intersect_full.txt
```   
Extract gene names and drop non-overlapping amplicons:

In [10]:
intersect_full = pd.read_csv('intersect_full.txt', sep='\t', header=None)
tmp = intersect_full[14].str.split(' ', expand=True).iloc[:, 9]. \
    reset_index().drop('index', axis=1).replace(np.nan, '.').replace([';', '"'], '', regex=True)
# drop column 14 for shorter view below
df = intersect_full.drop([14], axis=1)
tmp.columns = [14]
df = pd.concat([df, tmp], axis=1)
df = df.loc[df[14] != '.'].reset_index().drop('index', axis=1)

df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,chr1,113456395,113456721,AMPL7166063003,.,Pool=2;SUBMITTED_REGION=AMPL7166063003,chr1,ncbiRefSeq.2021-05-17,exon,113454469,113456787,.,-,.,SLC16A1
1,chr1,113456395,113456721,AMPL7166063003,.,Pool=2;SUBMITTED_REGION=AMPL7166063003,chr1,ncbiRefSeq.2021-05-17,3UTR,113454469,113456512,.,-,.,SLC16A1
2,chr1,113456395,113456721,AMPL7166063003,.,Pool=2;SUBMITTED_REGION=AMPL7166063003,chr1,ncbiRefSeq.2021-05-17,CDS,113456516,113456787,.,-,2,SLC16A1


In [11]:
len(df[14].unique())

31

2.2 We still have 31 genes. Let's **find out, which amplicons mapped on two or more genes**: 

In [12]:
def find_interesting_genes(df):
    ampls = df[3].unique()
    d = {}
    for ampl in ampls:
        set_genes = list(df.loc[df[3] == ampl][14].unique())
        if len(set_genes) > 1:
            d[ampl] = set_genes
    return d
find_interesting_genes(df)

{'AMPL7156752826': ['SLC16A1-AS1', 'SLC16A1'],
 'AMPL7154848931': ['GLIS3-AS1', 'GLIS3'],
 'AMPL7154784953': ['INS', 'INS-IGF2'],
 'AMPL7154784954': ['INS', 'INS-IGF2'],
 'AMPL7154784955': ['INS', 'INS-IGF2'],
 'AMPL7154897326': ['INS', 'INS-IGF2'],
 'AMPL7154511317': ['C12orf43', 'HNF1A'],
 'AMPL7154784910': ['C12orf43', 'HNF1A'],
 'AMPL7154784911': ['C12orf43', 'HNF1A'],
 'AMPL7154784912': ['C12orf43', 'HNF1A'],
 'AMPL7154783021': ['C12orf43', 'HNF1A'],
 'AMPL7159709172': ['C12orf43', 'HNF1A'],
 'AMPL7159709205': ['C12orf43', 'HNF1A']}

For next steps we take only these genes and amplicons:

In [13]:
tmp_d = find_interesting_genes(df)
tmp_df = pd.DataFrame()
for ampl in tmp_d.keys():
    tmp_df = pd.concat([tmp_df, df.loc[(df[3] == ampl) & df[14].isin(tmp_d[ampl])]], axis=0)

df = tmp_df.reset_index().drop('index', axis=1)

2.3 We already find information about them, but we can **check again, if any of genes are pseudogenes**. For this purpose we use [List] of pseudogenes in hg19. As we expected, there are no these genes in our target pannel.      
But we already can see some "strange" genes in the table above: chromosome 12 open reading frame 43 *C12orf43*, *GLIS3* antisense RNA 1, *SLC16A1* antisense RNA 1, *MFF* divergent transcript. They could be potential off-tartget reogions.   
     
2.4 **Search for homologous regions - 1st way**   
I have three ideas how to search how to search homologous or identical region, but only one I could implement. This is "manualy" method, and it is only suitable for searching for 100% homology.     
     
There are four possible position amplicon on target region (see the picture below): inside (<span style="color:blue">**blue**</span>), partly on left side (<span style="color:green">**green**</span>), partly on right side (<span style="color:yellow">**yellow**</span> ), target inside amplicon (<span style="color:purple">**purple**</span>).   
    
<img src="picture1.jpg" width="100%" height="50%"  />      
   
We can verify this by calculating the lengths of the overlaps, they will be different and often different from the length of the amplicon. If one amplicon mapped on two or more different genome region, we can we can consider 100% homologous overlap sequencies in three cases:   
- same left-side overlap  length  
- same left-side overlap length   
- same full overlap length  == amplicon lenght      
   
In <span style="color:purple">**purple**</span> it's impossible to find homologous regions.   

For further analysis we need to calculate amplicon and target region length, distance between amplicon and target region start positions, distance between amplicon and target region end positions:

[List]: http://genome.ucsc.edu/cgi-bin/hgSearch?search=pseudogene+mRNA&hgsid=1712866984_aQKAtsLa46m4JJkhGbsoT2fVyJFe&db=hg19

In [14]:
df['ampl_len'] = df[2] - df[1]
df['ref_len'] = df[10] - df[9]
df['ampl_start-ref_start'] = df[1] - df[9]
df['ref_end-ampl_end'] = df[10] - df[2]

df.head(6)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,ampl_len,ref_len,ampl_start-ref_start,ref_end-ampl_end
0,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,chr1,ncbiRefSeq.2021-05-17,exon,113499037,113499578,.,+,.,SLC16A1-AS1,323,541,-259,477
1,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,chr1,ncbiRefSeq.2021-05-17,exon,113499037,113500344,.,+,.,SLC16A1-AS1,323,1307,-259,1243
2,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,chr1,ncbiRefSeq.2021-05-17,exon,113497898,113498818,.,-,.,SLC16A1,323,920,880,-283
3,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,chr1,ncbiRefSeq.2021-05-17,5UTR,113497898,113498818,.,-,.,SLC16A1,323,920,880,-283
4,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,chr1,ncbiRefSeq.2021-05-17,exon,113498657,113498818,.,-,.,SLC16A1,323,161,121,-283
5,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,chr1,ncbiRefSeq.2021-05-17,5UTR,113498657,113498818,.,-,.,SLC16A1,323,161,121,-283


If `start_ampl-start_ref` > 0 and `stop_ref-stop_ampl` > 0, it's <span style="color:blue">**blue**</span> position (see picture above)   
If `start_ampl-start_ref` < 0 and `stop_ref-stop_ampl` > 0, it's <span style="color:green">**green**</span> position   
If `start_ampl-start_ref` > 0 and `stop_ref-stop_ampl` < 0, it's <span style="color:yellow">**yellow**</span> position    
If `start_ampl-start_ref` < 0 and `stop_ref-stop_ampl` < 0, it's <span style="color:purple">**purple**</span> position

Now let's calculate overlaps lengths, it will be negative in <span style="color:green">**green**</span> positions, in case of to amplicon with same overlap length in different regions

In [15]:
def overlap_length(df):
    if (df['ampl_start-ref_start'] >= 0) and (df['ref_end-ampl_end'] >= 0):
        return df['ampl_len']
    elif (df['ampl_start-ref_start'] < 0) and (df['ref_end-ampl_end'] > 0):
        return -(df['ampl_len'] + df['ampl_start-ref_start'])
    elif (df['ampl_start-ref_start'] > 0) and (df['ref_end-ampl_end'] < 0):
        return df['ampl_len'] + df['ref_end-ampl_end']
    elif (df['ampl_start-ref_start'] <= 0) and (df['ref_end-ampl_end'] <= 0):
        return np.nan


In [16]:
df['overlap_len'] = df.apply(lambda x: overlap_length(x), axis=1)
df = df.dropna().reset_index().drop('index', axis=1)

We can remove rows with the same amplicon ID, gene, part of gene and overlap length.    
Also we can drop rows where `overlap_len` is unique, because as we notice before, 100% homologu should have same length:

In [17]:
df = df.drop_duplicates([3, 8, 14, 'overlap_len'])

def drop_unique_length(df):
    tmp_list = []
    for x in df['overlap_len'].unique():
        n = list(df['overlap_len']).count(x)
        if n > 1:
            tmp_list.append(x)
    return df.loc[(df['overlap_len'].isin(tmp_list))].reset_index().drop('index', axis=1)


In [18]:
df = drop_unique_length(df)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,ampl_len,ref_len,ampl_start-ref_start,ref_end-ampl_end,overlap_len
0,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,chr1,ncbiRefSeq.2021-05-17,exon,113497898,113498818,.,-,.,SLC16A1,323,920,880,-283,40.0
1,chr1,113498778,113499101,AMPL7156752826,.,Pool=2;SUBMITTED_REGION=AMPL7156752826,chr1,ncbiRefSeq.2021-05-17,5UTR,113497898,113498818,.,-,.,SLC16A1,323,920,880,-283,40.0
2,chr11,2181834,2182073,AMPL7154784953,.,"Pool=1;SUBMITTED_REGION=AMPL7166063050,AMPL7154784954,AMPL7154784953",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182439,.,-,.,INS,239,424,-181,366,-58.0
3,chr11,2181834,2182073,AMPL7154784953,.,"Pool=1;SUBMITTED_REGION=AMPL7166063050,AMPL7154784954,AMPL7154784953",chr11,ncbiRefSeq.2021-05-17,CDS,2182015,2182201,.,-,0,INS,239,186,-181,128,-58.0
4,chr11,2181834,2182073,AMPL7154784953,.,"Pool=1;SUBMITTED_REGION=AMPL7166063050,AMPL7154784954,AMPL7154784953",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182218,.,-,.,INS-IGF2,239,203,-181,145,-58.0


Notice, that if we filter only exons, the set of "interesting" genes will be the same, so let's do it:

In [19]:
print(df.loc[df[8] == 'exon'][14].unique())
print(df[14].unique())

df = df.loc[df[8] == 'exon'].reset_index().drop('index', axis=1)

['SLC16A1' 'INS' 'INS-IGF2' 'C12orf43' 'HNF1A']
['SLC16A1' 'INS' 'INS-IGF2' 'C12orf43' 'HNF1A']


And drop rows where `overlap_len` is unique again:

In [20]:
df = drop_unique_length(df)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,ampl_len,ref_len,ampl_start-ref_start,ref_end-ampl_end,overlap_len
0,chr11,2181834,2182073,AMPL7154784953,.,"Pool=1;SUBMITTED_REGION=AMPL7166063050,AMPL7154784954,AMPL7154784953",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182439,.,-,.,INS,239,424,-181,366,-58.0
1,chr11,2181834,2182073,AMPL7154784953,.,"Pool=1;SUBMITTED_REGION=AMPL7166063050,AMPL7154784954,AMPL7154784953",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182218,.,-,.,INS-IGF2,239,203,-181,145,-58.0
2,chr11,2182062,2182385,AMPL7154784954,.,"Pool=2;SUBMITTED_REGION=AMPL7154784954,AMPL7154784953,AMPL7154784955",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182218,.,-,.,INS,323,203,47,-167,156.0
3,chr11,2182062,2182385,AMPL7154784954,.,"Pool=2;SUBMITTED_REGION=AMPL7154784954,AMPL7154784953,AMPL7154784955",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182218,.,-,.,INS-IGF2,323,203,47,-167,156.0
4,chr11,2182130,2182458,AMPL7154784955,.,"Pool=1;SUBMITTED_REGION=AMPL7154784955,AMPL7154897326,AMPL7154784954",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182218,.,-,.,INS,328,203,115,-240,88.0
5,chr11,2182130,2182458,AMPL7154784955,.,"Pool=1;SUBMITTED_REGION=AMPL7154784955,AMPL7154897326,AMPL7154784954",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182218,.,-,.,INS-IGF2,328,203,115,-240,88.0
6,chr11,2182429,2182753,AMPL7154897326,.,"Pool=2;SUBMITTED_REGION=AMPL7154784955,AMPL7154897326",chr11,ncbiRefSeq.2021-05-17,exon,2182015,2182439,.,-,.,INS,324,424,414,-314,10.0
7,chr11,2182429,2182753,AMPL7154897326,.,"Pool=2;SUBMITTED_REGION=AMPL7154784955,AMPL7154897326",chr11,ncbiRefSeq.2021-05-17,exon,2182398,2182439,.,-,.,INS-IGF2,324,41,31,-314,10.0
8,chr12,121438967,121439264,AMPL7154784910,.,"Pool=1;SUBMITTED_REGION=AMPL7154784910,AMPL7154511317,AMPL7154784911",chr12,ncbiRefSeq.2021-05-17,exon,121438289,121442292,.,-,.,C12orf43,297,4003,678,3028,297.0
9,chr12,121438967,121439264,AMPL7154784910,.,"Pool=1;SUBMITTED_REGION=AMPL7154784910,AMPL7154511317,AMPL7154784911",chr12,ncbiRefSeq.2021-05-17,exon,121438868,121440315,.,+,.,HNF1A,297,1447,99,1051,297.0


INS-IGF2 is a read-through gene, which overlaps with gene INS at the 5' region, so it's not off-target region.   
Apart from this, there are several amplicons, mapped on two different genes - *HNF1A* and *C12orf43*. Based on the descriptions of these genes (table in task1, file `RESULTS/genes_summary.csv`) I can assume, that *HNF1A* is a homologous off-target region.    
I could be important to notice, that these genes have similar coordinates in different DNA strand. 
   
**Target region coordinates:** 121438868, 121440315    
**Homologous off-target coordinates:** 121438289, 121442292    
   

Save results:

In [21]:
df = df.loc[df[14].isin(['HNF1A', 'C12orf43'])].reset_index().drop('index', axis=1)
df.to_csv('RESULTS/task2_homologous_region.csv', sep='\t', index=None)

*******************************

# 3. Two other ideas how to find homologous off-target region   
3.1 [**Segmental Duplication Database**](https://eeelegacy.gs.washington.edu/humanparalogy/build37/build37.htm)[[3]](https://pubmed.ncbi.nlm.nih.gov/11381028/)     

Download these database and try `bedtools intersect` again


In [22]:
sddb = pd.read_excel("build37.xlsx")

In [23]:
sddb

,chrom,chromStart,chromEnd,name,score,strand,otherChrom,otherStart,otherEnd,otherSize,...,indelS,alignB,matchB,mismatchB,transitionsB,transversionsB,fracMatch,fracMatchIndel,jcK,k2K
0,chr1,85326,87112,chr1:398212,0,_,chr1,398212,400000,1788,...,2,1786,1757,29,15,14,0.983763,0.982662,0.016416,0.016423
1,chr1,398212,400000,chr1:85326,0,_,chr1,85326,87112,1786,...,2,1786,1757,29,15,14,0.983763,0.982662,0.016416,0.016423
2,chr1,88000,121417,chr1:235525,0,+,chr1,235525,267707,32182,...,1299,32150,31941,209,133,76,0.993499,0.992727,0.006529,0.006532
3,chr1,235525,267707,chr1:88000,0,+,chr1,88000,121417,33417,...,1299,32150,31941,209,133,76,0.993499,0.992727,0.006529,0.006532
4,chr1,91256,92392,chr1:521369,0,+,chr1,521369,522487,1118,...,20,1117,1092,25,18,7,0.977619,0.974130,0.022722,0.022781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51594,chrY,58880431,58891092,chrY:58869728,0,+,chrY,58869728,58880430,10702,...,99,10632,10566,66,25,41,0.993792,0.992485,0.006234,0.006234
51595,chrY,58881608,58885137,chrY:58885137,0,+,chrY,58885137,58888721,3584,...,55,3529,3509,20,9,11,0.994333,0.993488,0.005689,0.005689
51596,chrY,58885137,58888721,chrY:58881608,0,+,chrY,58881608,58885137,3529,...,55,3529,3509,20,9,11,0.994333,0.993488,0.005689,0.005689
51597,chrY,59012496,59013612,chrY:59024857,0,_,chrY,59024857,59025974,1117,...,3,1115,1041,74,42,32,0.933632,0.931961,0.069490,0.069706


Extract 4-column BED-file:

In [24]:
sddb.rename(columns={'chrom':'#chrom'})
sddb_short = sddb.iloc[:, [0, 1, 2, 3]]
sddb_short.to_csv('sddb.bed', index=None, sep='\t')

Run `bedtools intersect` with SDDB BED-file and output BED-file from tak1 (to have genes names):   
```bash
bedtools intersect -a ./RESULTS/task1_output.bed -b sddb.bed -wa -wb > sddb_intersect.txt
```

In [25]:
pd.read_csv('sddb_intersect.txt', sep='\t', header=None)

,0,1,2,3,4,5,6,7,8,9,10,11
0,chr9,135937315,135937599,AMPL7154848171,.,Pool=1;SUBMITTED_REGION=AMPL7154848171,CEL,1,chr9,135935536,135947286,chr9:135956012
1,chr9,135939554,135939855,AMPL7154785491,.,"Pool=2;SUBMITTED_REGION=AMPL7154785491,AMPL7162958722",CEL,2,chr9,135935536,135947286,chr9:135956012
2,chr9,135939844,135940096,AMPL7162958722,.,"Pool=1;SUBMITTED_REGION=AMPL7154783210,AMPL7154785491,AMPL7162958722",CEL,2,chr9,135935536,135947286,chr9:135956012
3,chr9,135939844,135940096,AMPL7162958722,.,"Pool=1;SUBMITTED_REGION=AMPL7154783210,AMPL7154785491,AMPL7162958722",CEL,3,chr9,135935536,135947286,chr9:135956012
4,chr9,135940061,135940234,AMPL7154783210,.,"Pool=2;SUBMITTED_REGION=AMPL7154785486,AMPL7154783210,AMPL7162958722",CEL,3,chr9,135935536,135947286,chr9:135956012
5,chr9,135940223,135940552,AMPL7154785486,.,"Pool=1;SUBMITTED_REGION=AMPL7154785486,AMPL7154783210,AMPL7154848184",CEL,4,chr9,135935536,135947286,chr9:135956012
6,chr9,135940524,135940722,AMPL7154848184,.,"Pool=2;SUBMITTED_REGION=AMPL7154848184,AMPL7154785486",CEL,4,chr9,135935536,135947286,chr9:135956012
7,chr9,135941786,135942083,AMPL7154848187,.,Pool=1;SUBMITTED_REGION=AMPL7154848187,CEL,5,chr9,135935536,135947286,chr9:135956012
8,chr9,135942100,135942411,AMPL7154848190,.,"Pool=2;SUBMITTED_REGION=AMPL7154848193,AMPL7154848190",CEL,6,chr9,135935536,135947286,chr9:135956012
9,chr9,135942385,135942705,AMPL7154848193,.,"Pool=1;SUBMITTED_REGION=AMPL7154848190,AMPL7154848193",CEL,7,chr9,135935536,135947286,chr9:135956012


We see, that regions in chr10 have duplicates in chrX, for example:

In [26]:
sddb.loc[(sddb['chrom'] == 'chr10') & (sddb['chromStart'] == 88810248)]

,chrom,chromStart,chromEnd,name,score,strand,otherChrom,otherStart,otherEnd,otherSize,...,indelS,alignB,matchB,mismatchB,transitionsB,transversionsB,fracMatch,fracMatchIndel,jcK,k2K
8705,chr10,88810248,88811628,chrX:120183094,0,_,chrX,120183094,120184459,1365,...,31,1357,1308,49,36,13,0.963891,0.95684,0.037007,0.037179


But there are indels. Perhaps, with a more detailed approach, this database could be useful in searching homologous off-target regions.   

3.2 **Special tools for searching homologous regions**   
There are some tools and pipelines used for searching homologous region, such as [i-ADHoRe](https://academic.oup.com/nar/article/40/2/e11/2409742), [FASTA](https://www.ebi.ac.uk/Tools/sss/fasta/nucleotide.html) or [this algorithm](https://www.nature.com/articles/gim201658)   
   
All of them work with fasta files, we can get it easily from BED-file.   
First of all, we need to get fasta file from input BED-file.  
    
Download reference genome hg19:   
```bash
wget ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/chromosomes/*
for file in ./*.gz; do gunzip $file; done
cat *.fa > hg19.genome.fa
```
   
Getting fasta from BED:   
```bash
bedtools getfasta -fi hg19.genome.fa -bed IAD143293_241_Designed.bed > input.fasta
```
   
I am not testing these tools, but I assume, if we understand their work well, we can adapt them to our needs


*******************